# Projet PIST VieML
**Video Information Extractor by Machine Learning**

*Thomas Schillaci, Léo Vuylsteker, François Le Guernic, Quentin Seite, Yu Teng, Aymeric Varasse, Alejo Velez, Mehdi Rifaï*

# 6 Mars :
- On décide d'utiliser Keras comme libraire de ML car celle-ci rend le développement plus facile, en effet elle propose une API ergonomique qui vient se greffer sur plusieurs librairies au choix (Tensorflow, Pytorch ...)
- On décide d'utiliser Notebook pour suivre l'évolution du développement
- On décide d'utiliser Github pour partager le projet entre nous

# 13 Mars :
- Premier programme de ML capable de simuler une porte ET

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

model = Sequential()
model.add(Dense(16, activation='relu', input_dim=2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])

x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], "float32")
y = np.array([[0], [0], [0], [1]], "float32")

model.fit(x, y, epochs=200)

y_predict = model.predict(x).round()
print(y_predict)

# 20 Mars :
- Premier réseau neuronal convolutif
- Collection de datasets : http://deeplearning.net/datasets/
- Datasets d'objets sous différents angles de vue :
	- http://www.cs.columbia.edu/CAVE/software/softlib/coil-20.php\
	- http://www1.cs.columbia.edu/CAVE/software/softlib/coil-100.php

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32') / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation="relu", data_format="channels_first"))
model.add(MaxPooling2D())
model.add(Conv2D(64, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=200, epochs=10, validation_data=(x_test, y_test))

print(model.evaluate(x_test, y_test))

# 27 Mars :
Le groupe ML est séparé en deux équipes en compétition
- Création d'un réseau neuronal convolutif recréant une image à partir de plusieurs angles de vues
- Création d'un réseau neuronal d'upscaling d'image

# 3 Avril :
- La première hyperparamétrisation du réseau multi-angle a donné les résultats suivants :
	- optimizer : Adam
	- batch size : 2
	- loss : mean_squared_error
	- dense size : 1024
	
# 1er Mai :
Implémentation de l'optimisateur L-BFGS
Résultats peu fructueux, cet optimisateur converge très lentement par rapport à Adam :

In [ ]:
import numpy as np
from scipy.optimize import minimize
from keras.models import Sequential
from keras.layers.core import Dense

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [0], [0], [1]])

model = Sequential()
model.add(Dense(4, activation='sigmoid', input_dim=2))
model.add(Dense(4, activation='sigmoid', input_dim=2))
model.add(Dense(1, activation='sigmoid', input_dim=2))

model.compile("Adam", "binary_crossentropy", metrics=["accuracy"])
# model.fit(X, Y, epochs=3000)

index = 0


def loss(W):
    global model, index
    weights = model.get_weights()
    convert(weights, W)
    index = 0
    model.set_weights(weights)
    error = model.evaluate(X, Y)[0]
    print(error)
    return error


def convert(weights, W):
    global index
    for i in range(len(weights)):
        e = weights[i]
        if isinstance(e, np.ndarray):
            convert(e, W)
        else:
            weights[i] = W[index]
            index += 1


tmp = np.array(model.get_weights())
x0 = []
for i in range(tmp.shape[0]):
    x0 = np.append(x0, tmp[i].flatten())
loss(x0)
res = minimize(loss, x0, method='L-BFGS-B', options={'eps': 1e-3, 'disp': True})
print(model.predict(X).round())

# 8 Mai :
Nouvelle hyper-paramétrisation du réseau multi-angle suite à ses améliorations successives :
    - optimizer: 
    - learning rate:
    - batch size: 
    - dense size:
    - dropout rate: